In [80]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, LeaveOneOut,\
RepeatedKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [81]:
import warnings
warnings.filterwarnings("ignore")

In [82]:
df = pd.read_csv('data_split')

df.isna().sum()

In [83]:
df.fillna('missing', inplace=True)

df.isna().sum()

In [51]:
X = df.drop('Target',axis=1)
y = df['Target']

In [52]:
X.head()

,Unnamed: 0,Variable_1,Variable_2,Variable_3,Variable_4,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,...,Variable_34,Variable_35,Variable_36,Variable_37,Variable_39,Variable_40,Variable_41,Variable_5,Variable_14,Variable_45
0,0,35.0,1.0,1.0,1.0,72.0,27.0,100.0,190.0,100.0,...,0.0,0.0,0.0,17550.5,699.0,0.0,0.0,0,11.0,1
1,1,30.0,1.0,1.0,1.0,25.0,26.0,199.0,199.0,199.0,...,0.0,0.0,0.0,40147.0,100.0,0.0,0.0,0,11.0,2
2,2,30.0,1.0,1.0,1.0,65.0,21.0,100.0,100.0,100.0,...,0.0,0.0,0.0,499.0,300.0,0.0,0.0,0,6.0,1
3,3,30.0,1.0,1.0,1.0,65.0,48.0,199.0,199.0,199.0,...,0.0,0.0,0.0,17550.5,699.0,0.0,0.0,0,3.0,1
4,4,30.0,2.0,1.0,1.0,4.0,28.0,199.0,600.0,199.0,...,0.0,0.0,0.0,17550.5,699.0,0.0,0.0,0,6.0,1


In [53]:
X.shape

(2057, 40)

In [54]:
y.value_counts()

1.0    1424
0.0     633
Name: Target, dtype: int64

In [132]:
kf = KFold(n_splits=10)

In [133]:
i = 1
for train_set, test_set in kf.split(X=X):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print("-------------------------")
    i += 1

iteration  1
[ 206  207  208 ... 2054 2055 2056]  having : 1851
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205]  having : 206
-------------------------
iteration  2
[   0    1    2 ... 2054 2055 2056]  having : 

In [134]:
num_cols = X.select_dtypes(include=np.number).columns
num_cols

Index(['Unnamed: 0', 'Variable_1', 'Variable_2', 'Variable_3', 'Variable_4',
       'Variable_6', 'Variable_7', 'Variable_8', 'Variable_9', 'Variable_10',
       'Variable_11', 'Variable_12', 'Variable_15', 'Variable_16',
       'Variable_17', 'Variable_18', 'Variable_19', 'Variable_20',
       'Variable_22', 'Variable_23', 'Variable_24', 'Variable_25',
       'Variable_26', 'Variable_27', 'Variable_28', 'Variable_29',
       'Variable_30', 'Variable_31', 'Variable_32', 'Variable_33',
       'Variable_34', 'Variable_35', 'Variable_36', 'Variable_37',
       'Variable_39', 'Variable_40', 'Variable_41', 'Variable_5',
       'Variable_14', 'Variable_45'],
      dtype='object')

In [135]:
cat_cols = X.select_dtypes(exclude=np.number).columns
cat_cols

Index([], dtype='object')

In [136]:
X[cat_cols]=X[cat_cols].astype('float')

ct = ColumnTransformer([
    ('rob', RobustScaler(), num_cols),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'), cat_cols)
])

In [171]:
import lightgbm as lgb
model = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.15, max_depth=3,
               min_child_samples=20, min_child_weight=1, min_split_gain=0.0,
               n_estimators=40, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')

pipe = Pipeline([
    ('ct_step', ct),
    ('model', RandomForestClassifier(n_estimators=10, random_state=0))
])

In [175]:
pipe = Pipeline([
    ('ct_step', ct),
    ('model', model)
])

In [176]:
X.loc[2056:].head()

,Unnamed: 0,Variable_1,Variable_2,Variable_3,Variable_4,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,...,Variable_34,Variable_35,Variable_36,Variable_37,Variable_39,Variable_40,Variable_41,Variable_5,Variable_14,Variable_45
2056,2056,30.0,1.0,1.0,1.0,50.0,34.0,0.0,500.0,100.0,...,0.0,0.0,0.0,12326.0,450.0,0.0,0.0,0,11.0,2


In [177]:
scores = []
i = 1
for train_set, test_set in kf.split(X):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [178]:
np.array(scores)

array([0.62621359, 0.69902913, 0.67961165, 0.72815534, 0.73786408,
       0.7184466 , 0.68446602, 0.70243902, 0.65853659, 0.71219512])

In [179]:
np.array(scores).mean()

0.6946957139474307

In [180]:
np.array(scores).std()

0.032033708282889733

# Stratified KFold

In [181]:
skf = StratifiedKFold(n_splits=10)

In [182]:
scores_skf = []
i = 1
for train_set, test_set in skf.split(X, y):
    pipe.fit(X.loc[train_set], y[train_set])
    sco = pipe.score(X.loc[test_set], y[test_set])
    scores_skf.append(sco)
    print("iteration ", i)
    i += 1

iteration  1
iteration  2
iteration  3
iteration  4
iteration  5
iteration  6
iteration  7
iteration  8
iteration  9
iteration  10


In [183]:
scores_skf

[0.4320388349514563,
 0.6699029126213593,
 0.6990291262135923,
 0.6941747572815534,
 0.6893203883495146,
 0.6893203883495146,
 0.6310679611650486,
 0.7024390243902439,
 0.6487804878048781,
 0.7121951219512195]

In [184]:
i = 1
for train_set, test_set in skf.split(X=X, y=y):
    print("iteration ", i)
    print(train_set, " having :" , len(train_set))
    print(test_set, " having :" , len(test_set))
    print()
    print("y train counts: \n", y[train_set].value_counts())
    print("y test counts: \n", y[test_set].value_counts())
    print("-------------------------")
    i += 1

iteration  1
[ 182  183  185 ... 2054 2055 2056]  having : 1851
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 184 187 188 189 190 192 193 194 195 196 197 198 199 202 204 205
 206 207 210 211 212 213 214 215]  having : 206

y train counts: 
 1.0    1281
0.0     570
Name: Target, dtype: int64
y test counts: 

In [185]:
result_kf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=10)

In [186]:
result_kf

array([0.43203883, 0.66990291, 0.69902913, 0.69417476, 0.68932039,
       0.68932039, 0.63106796, 0.70243902, 0.64878049, 0.71219512])

In [187]:
start = time.time()
result_kf10 = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy', cv=KFold(n_splits=10))
result_kf10
print("time taken: ", time.time()-start)

time taken:  0.28324222564697266


In [188]:
result_kf10

array([0.62621359, 0.69902913, 0.67961165, 0.72815534, 0.73786408,
       0.7184466 , 0.68446602, 0.70243902, 0.65853659, 0.71219512])

# LOO CV

In [189]:
start = time.time()
result_loocv = cross_val_score(estimator=pipe, X=X.head(100), y=y.head(100),
                               scoring='accuracy', cv=LeaveOneOut())
print("time taken: ", time.time()-start)

time taken:  1.9098939895629883


In [190]:
result_loocv

array([0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.])

In [191]:
result_loocv.mean()

0.73

# Repeated KFold

In [200]:
start = time.time()
result_rkf = cross_val_score(estimator=pipe, X=X, y=y, scoring='accuracy',
                              cv=RepeatedKFold(n_splits=10, n_repeats=10))
result_rkf
print("time taken: ", time.time()-start)

time taken:  2.778571605682373


In [201]:
result_rkf

array([0.66990291, 0.7184466 , 0.66990291, 0.7184466 , 0.75728155,
       0.7184466 , 0.63592233, 0.66341463, 0.67804878, 0.67804878,
       0.70873786, 0.7184466 , 0.7038835 , 0.61165049, 0.70873786,
       0.66019417, 0.71359223, 0.67317073, 0.71707317, 0.69268293,
       0.72815534, 0.72815534, 0.65533981, 0.69902913, 0.72815534,
       0.7038835 , 0.65533981, 0.64390244, 0.68780488, 0.65853659,
       0.63592233, 0.69902913, 0.66990291, 0.65533981, 0.69417476,
       0.66504854, 0.73300971, 0.66341463, 0.72682927, 0.71219512,
       0.70873786, 0.62621359, 0.66990291, 0.69417476, 0.71359223,
       0.69417476, 0.64563107, 0.71219512, 0.73658537, 0.68292683,
       0.70873786, 0.75728155, 0.66504854, 0.62135922, 0.72330097,
       0.71359223, 0.71359223, 0.63414634, 0.69268293, 0.64390244,
       0.69417476, 0.70873786, 0.67475728, 0.7038835 , 0.7038835 ,
       0.66990291, 0.68446602, 0.70243902, 0.68780488, 0.70243902,
       0.71359223, 0.68932039, 0.66504854, 0.68932039, 0.66990

In [202]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [203]:
start = time.time()
result_tts = cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='accuracy', cv=KFold(n_splits=10))
print("time taken: ", time.time()-start)

time taken:  0.2603030204772949


In [204]:
result_tts

array([0.68484848, 0.6969697 , 0.7030303 , 0.63636364, 0.69090909,
       0.7195122 , 0.73780488, 0.68902439, 0.71341463, 0.66463415])

In [205]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('ct_step',
                 ColumnTransformer(transformers=[('rob', RobustScaler(),
                                                  Index(['Unnamed: 0', 'Variable_1', 'Variable_2', 'Variable_3', 'Variable_4',
       'Variable_6', 'Variable_7', 'Variable_8', 'Variable_9', 'Variable_10',
       'Variable_11', 'Variable_12', 'Variable_15', 'Variable_16',
       'Variable_17', 'Variable_18', 'Variable_19', 'Variable_20',
       'Variable_22', 'Varia...
       'Variable_34', 'Variable_35', 'Variable_36', 'Variable_37',
       'Variable_39', 'Variable_40', 'Variable_41', 'Variable_5',
       'Variable_14', 'Variable_45'],
      dtype='object')),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  Index([], dtype='object'))])),
                ('model',
 

In [206]:
pipe.score(X_test, y_test)

0.6771844660194175

In [216]:
cross_val_score(estimator=pipe, X=X_train, y=y_train,
                              scoring='roc_auc', cv=KFold(n_splits=100)).mean()

0.6659640914640914

import sklearn
sorted(sklearn.metrics.SCORERS.keys())